In [1]:
# ЯЧЕЙКА 2 — Импорт и загрузка моделей (всё скачается само при первом запуске)
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss, pandas as pd, numpy as np
from tqdm.auto import tqdm
import os

print("Загружаем модели... (первый раз будет 1–2 минуты)")

# ← ЛУЧШИЕ модели 2025 года для русского языка и поиска
bi_encoder_raw   = SentenceTransformer('deepvk/RuModernBERT-small')   # Bi-Encoder (быстрый поиск)
cross_encoder_raw = CrossEncoder('DiTy/cross-encoder-russian-msmarco')                 # Cross-Encoder (точный реранкинг)

print("Модели готовы!")

KeyboardInterrupt: 

In [2]:
# ЯЧЕЙКА 3 — Твои данные (пока просто пример, потом заменишь на свои)
# Формат: csv файл с колонками: post_text, channel_name
data = [
    {"post_text": "Девочки, кто брал зимнюю куртку Reima на рост 104? Как греет?", "channel_name": "Мамочки Москвы"},
    {"post_text": "Продам коляску Anex m/type 3в1 в идеале, 25к", "channel_name": "Мамочки Москвы"},
    {"post_text": "Ищу хорошие кроссовки Nike на подростка 38 размер", "channel_name": "Мамочки Москвы"},
    {"post_text": "Reima, Lassie, Kerry — обзор зимних комбезов 2024", "channel_name": "Детская одежда СПб"},
    {"post_text": "Продам iPhone 15 Pro Max титановый, новый", "channel_name": "Гаджеты и техника"},
    {"post_text": "Кто шил зимний комплект в ателье? Цены космос", "channel_name": "Детская одежда СПб"},
]

df = pd.DataFrame(data)
print(f"Загружено {len(df)} постов из {df.channel_name.nunique()} каналов")

Загружено 6 постов из 3 каналов


In [4]:
# ЯЧЕЙКА 4 — Один раз: превращаем все посты в векторы и строим индекс
print("Считаем векторы всех постов...")

post_embeddings = bi_encoder_raw.encode(
    df.post_text.tolist(),
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True
)

dimension = post_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(post_embeddings)

# Сохраняем для будущего (чтобы не пересчитывать)
faiss.write_index(index, "posts_index.faiss")
df.to_csv("posts.csv", index=False)

print(f"Индекс построен! {index.ntotal} постов")

Считаем векторы всех постов...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Индекс построен! 6 постов


In [5]:
# ЯЧЕЙКА 6 — ТЕСТ!
from recommend import recommend_channels


query = "Зимняя куртка Reima на рост 104, водонепроницаемая"

print(f"\nЗапрос: {query}\n")
results = recommend_channels(
    query="Зимняя куртка Reima 104",
    bi_encoder=bi_encoder_raw,        # ← модель из коробки
    cross_encoder=cross_encoder_raw,
    index=index,
    df=df,
    top_k_posts=100,
    top_k_channels=10
)

for i, (channel, score) in enumerate(results, 1):
    print(f"{i}. {channel} → {score:.3f}")


Запрос: Зимняя куртка Reima на рост 104, водонепроницаемая

1. Мамочки Москвы → 0.323
2. Детская одежда СПб → 0.019
3. Гаджеты и техника → 0.001


In [6]:
# ЯЧЕЙКА — Сохраняем модель из коробки для сравнения
bi_encoder_raw = bi_encoder_raw  # это у тебя уже есть в памяти

# Сохраняем в папку рядом с ноутбуком
bi_encoder_raw.save("bi_encoder_rawT")
cross_encoder_raw.save("cross_encoder_raw")

print("Модель из коробки сохранена в папку: model_raw_RuModernBERT")
print("Теперь её можно использовать в любом ноутбуке так:")
print('SentenceTransformer("model_raw_RuModernBERT")')

Модель из коробки сохранена в папку: model_raw_RuModernBERT
Теперь её можно использовать в любом ноутбуке так:
SentenceTransformer("model_raw_RuModernBERT")


In [9]:
import pandas as pd
data = pd.read_csv('apps_channels (1)_test.csv', 
                   encoding='cp1251',
                   sep=';', 
                   on_bad_lines='skip')
data.head(100)

,id,category,country,channel,handle,name,description,image,subscribers
0,1,apps,Global,https://t.me/coubnews,@coubnews,Coub.com News,"Coub: Watch, Create, Earn!",https://static8.tgstat.ru/channels/_100/7c/7ce...,2759386
1,2,apps,RU,https://t.me/TikTokModCloud,@TikTokModCloud,TikTokModCloud,№ 4776856073,https://static7.tgstat.ru/channels/_100/23/237...,1796424
2,3,apps,Global,https://t.me/AssemBlogger,@AssemBlogger,??????? ?????? ??????,??? ?????? ?? ????? ????? ????????? ????? ????...,https://static7.tgstat.ru/channels/_100/ce/cea...,1404952
3,4,apps,IR,https://t.me/irproxy,@irproxy,?????? ??? | ?? ?? ?? ???,FAST MTPROTO PROXIES FOR TELEGRAM,https://static3.tgstat.ru/channels/_100/73/730...,1288779
4,5,apps,RU,https://t.me/EasyAPK,@EasyAPK,?? Easy APK,Бесплатные игры и программы для Android,https://static3.tgstat.ru/channels/_100/d2/d29...,1150436
...,...,...,...,...,...,...,...,...,...
95,98,apps,RU,https://t.me/blinkmap,@blinkmap,Blink HQ — твои друзья на карте,"новости, апдейты, мемы",https://static6.tgstat.ru/channels/_100/96/963...,177550
96,99,apps,Global,https://t.me/Windows_Softwares_Pc_Games,@Windows_Softwares_Pc_Games,Windows Softwares pc Games,Cracked Software's,https://static6.tgstat.ru/channels/_100/aa/aa7...,173839
97,100,apps,Global,https://t.me/key_outline,@key_outline,Outline Keys | Open | Free,Outline Keys Every day For Free,https://static1.tgstat.ru/channels/_100/9d/9d5...,173728
98,101,apps,Global,https://t.me/DramaLive7,@DramaLive7,Drama Live,IPTV Player Application | DramaLive.App,https://static10.tgstat.ru/channels/_100/6f/6f...,171540
